In [90]:
import onnxruntime as rt
import torch
from coco_eval import CocoEvaluator
from tqdm.notebook import tqdm

import numpy as np
import onnx
from onnxruntime import quantization
from onnxruntime.quantization import quantize_dynamic, QuantType

In [91]:
yolon_path = "yolon_best.onnx"
nanodet_path = "nanodet.onnx"

In [92]:
from roboflow import Roboflow
rf = Roboflow(api_key="xrjb7ahPJgT610pkOEf2")
project = rf.workspace("mooncrater").project("mooncrater")
dataset = project.version(3).download("coco")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to mooncrater-3 in coco:: 100%|██████████| 1587/1587 [00:00<00:00, 7921.75it/s]


In [93]:
import os
import torchvision

# settings
ANNOTATION_FILE_NAME = "_annotations.coco.json"
TRAIN_DIRECTORY = os.path.join(dataset.location, "train")
VAL_DIRECTORY = os.path.join(dataset.location, "valid")
TEST_DIRECTORY = os.path.join(dataset.location, "test")

class CocoDetection(torchvision.datasets.CocoDetection):
    def __init__(
        self,
        image_directory_path: str,
        # onnx_model_path: str,
        train: bool = True
    ):
        annotation_file_path = os.path.join(image_directory_path, ANNOTATION_FILE_NAME)
        super(CocoDetection, self).__init__(image_directory_path, annotation_file_path)

    def __getitem__(self, idx):
        images, annotations = super(CocoDetection, self).__getitem__(idx)
        image_id = self.ids[idx]
        annotations = {'image_id': image_id, 'annotations': annotations}
        
        return images, annotations


TRAIN_DATASET = CocoDetection(
    image_directory_path=TRAIN_DIRECTORY,
    train=True)
VAL_DATASET = CocoDetection(
    image_directory_path=VAL_DIRECTORY,
    train=False)
TEST_DATASET = CocoDetection(
    image_directory_path=TEST_DIRECTORY,
    train=False)

print("Number of training examples:", len(TRAIN_DATASET))
print("Number of validation examples:", len(VAL_DATASET))
print("Number of test examples:", len(TEST_DATASET))

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Number of training examples: 1479
Number of validation examples: 80
Number of test examples: 20


In [68]:
class QuantizationDataReader(quantization.CalibrationDataReader):
    def __init__(self, test_ds, batch_size, input_name):

        self.test_ds = test_ds

        self.input_name = input_name
        self.datasize = len(self.test_ds)

        self.enum_data = iter(self.test_ds)

    def to_numpy(self, image):
        image = np.array(image)
        image = image.astype(np.float32) / 255.0
        # print(image)
        image = np.transpose(image, (2, 0, 1))  # Change data layout from HWC to CHW
        image = np.expand_dims(image, axis=0) 

        return image

    def get_next(self):
        batch = next(self.enum_data, None)
        if batch is not None:
          return {self.input_name: self.to_numpy(batch[0])}
        else:
          return None

    def rewind(self):
        self.enum_data = iter(self.torch_dl)



In [87]:
TEST_DATASET[2]

(<PIL.Image.Image image mode=RGB size=640x640>,
 {'image_id': 2,
  'annotations': [{'id': 7,
    'image_id': 2,
    'category_id': 3,
    'bbox': [74, 238, 35, 43.5],
    'area': 1522.5,
    'segmentation': [],
    'iscrowd': 0},
   {'id': 8,
    'image_id': 2,
    'category_id': 3,
    'bbox': [489, 261, 33.5, 24.5],
    'area': 820.75,
    'segmentation': [],
    'iscrowd': 0},
   {'id': 9,
    'image_id': 2,
    'category_id': 3,
    'bbox': [3, 304, 49, 61.5],
    'area': 3013.5,
    'segmentation': [],
    'iscrowd': 0},
   {'id': 10,
    'image_id': 2,
    'category_id': 3,
    'bbox': [397, 324, 68.5, 50],
    'area': 3425,
    'segmentation': [],
    'iscrowd': 0},
   {'id': 11,
    'image_id': 2,
    'category_id': 3,
    'bbox': [114, 344, 48.5, 30.5],
    'area': 1479.25,
    'segmentation': [],
    'iscrowd': 0},
   {'id': 12,
    'image_id': 2,
    'category_id': 3,
    'bbox': [43, 346, 56.5, 52],
    'area': 2938,
    'segmentation': [],
    'iscrowd': 0},
   {'id': 13,


In [70]:

onnx_yolon = rt.InferenceSession(yolon_path)


In [71]:
from PIL import ImageDraw

In [100]:
import time
def inf_on_image(input_pil_image, onnx_model):
    
    image = np.array(input_pil_image)

    # Preprocess the images as required by your ONNX model
    # This might involve resizing, normalization, adding a batch dimension, etc.
    # The exact preprocessing steps will depend on your specific model

    # Convert the images to a format that can be used as input to the ONNX model
    # print(image)
    image = image.astype(np.float32) / 255.0
    # print(image)
    image = np.transpose(image, (2, 0, 1))  # Change data layout from HWC to CHW
    image = np.expand_dims(image, axis=0)  # Add batch dimension

    # Run the ONNX model
    input_name = onnx_model.get_inputs()[0].name

    start = time.perf_counter()
    predictions = onnx_model.run(None, {input_name: image})
    end = (time.perf_counter() - start) * 1000

    return predictions, end

def draw_preds(input_pil_image, predictions, conf=0.70):
    draw = ImageDraw.Draw(input_pil_image)

    # Define a color for each label
    colors = {0: "green", 1: "blue", 2: "red"}

    for prediction in predictions[0][0]:

        x1, y1, x2, y2, score, label = prediction
        
        #print(score, label)

        if score > conf:
            # Draw the bounding box on the image
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            draw.rectangle([x1, y1, x2, y2], outline=colors[int(label)], width=2)

    # Display the image
    input_pil_image.show()
    

In [73]:
test_img, _ = TEST_DATASET[4]

preds = inf_on_image(test_img)

print(preds[0].shape)

draw_preds(test_img, preds, 0.25)

TypeError: inf_on_image() missing 1 required positional argument: 'onnx_model'

In [74]:
TEST_DATASET.coco.dataset['annotations']

[{'id': 0,
  'image_id': 0,
  'category_id': 2,
  'bbox': [100, 402, 110.5, 38.5],
  'area': 4254.25,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 1,
  'image_id': 1,
  'category_id': 2,
  'bbox': [41, 303, 100.5, 29.5],
  'area': 2964.75,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 2,
  'image_id': 1,
  'category_id': 2,
  'bbox': [510, 330, 123.5, 46],
  'area': 5681,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 3,
  'image_id': 1,
  'category_id': 2,
  'bbox': [342, 339, 65, 33.5],
  'area': 2177.5,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 4,
  'image_id': 1,
  'category_id': 2,
  'bbox': [55, 361, 91.5, 34.5],
  'area': 3156.75,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 5,
  'image_id': 1,
  'category_id': 2,
  'bbox': [291, 396, 262.5, 79],
  'area': 20737.5,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 6,
  'image_id': 1,
  'category_id': 2,
  'bbox': [4, 427, 218.5, 50.5],
  'area': 11034.25,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 7,
  'image_id': 2,


In [127]:
import time

def calculate_metrics(inference_func, session, scale=1):
    evaluator = CocoEvaluator(coco_gt=TEST_DATASET.coco, iou_types=["bbox"])

    print("Running evaluation...")

    total_model_time = 0.0

    for idx, (input_pil_image, annotation) in enumerate(TEST_DATASET):
        
        image_id = annotation['image_id']

        predictions, elapsed_time = inference_func(input_pil_image, session)
        total_model_time += elapsed_time
        coco_results = []
        for prediction in predictions[0][0]:

            # Extract the bounding box coordinates, confidence score, and class label
            x1, y1, x2, y2, score, label = prediction
            x1, y1, x2, y2 = x1*scale, y1*scale, x2*scale, y2*scale
            # Convert the bounding box from (x1, y1, x2, y2) to (x, y, width, height)
            box = [x1, y1, x2 - x1, y2 - y1]

            print(score,label)

            # if score > 0.25:
                # Append the result
            coco_results.append({
                "image_id": image_id,
                "category_id": int(label)+1,
                "bbox": box,
                "score": float(score),
            })
        
        if len(coco_results) > 0:
            evaluator.update(coco_results)
    


    print(f"Elapsed time: {total_model_time} seconds")
    

    evaluator.synchronize_between_processes()
    evaluator.accumulate()
    evaluator.summarize()

In [39]:
import json
resfile = "results_nanodet.json"

with open(resfile, 'r') as f:
    nanodet_data = json.load(f)

In [6]:
nanodet_data[:10]

[{'image_id': 0,
  'category_id': 1,
  'bbox': [96.33306884765625,
   404.11553955078125,
   118.45657348632812,
   40.0133056640625],
  'score': 0.3991802930831909},
 {'image_id': 0,
  'category_id': 1,
  'bbox': [59.22003173828125,
   565.638671875,
   99.22882080078125,
   41.3145751953125],
  'score': 0.24268300831317902},
 {'image_id': 0,
  'category_id': 1,
  'bbox': [145.38497924804688,
   473.3541564941406,
   63.167022705078125,
   29.90277099609375],
  'score': 0.22241920232772827},
 {'image_id': 0,
  'category_id': 1,
  'bbox': [95.2303466796875,
   409.7038269042969,
   94.34355163574219,
   30.459381103515625],
  'score': 0.20320825278759003},
 {'image_id': 0,
  'category_id': 1,
  'bbox': [37.54389953613281,
   402.4532775878906,
   168.1042022705078,
   37.561767578125],
  'score': 0.1933024376630783},
 {'image_id': 0,
  'category_id': 1,
  'bbox': [211.1273651123047,
   368.3793640136719,
   82.35832214355469,
   21.960601806640625],
  'score': 0.17917358875274658},
 {'

In [40]:
for item in nanodet_data:
    item['category_id'] += 1

In [29]:
nanodet_data

[{'image_id': 0,
  'category_id': 1,
  'bbox': [215.38153076171875,
   395.5082092285156,
   363.05535888671875,
   244.49179077148438],
  'score': 0.06185806542634964,
  'segmentation': [[215.38153076171875,
    395.5082092285156,
    215.38153076171875,
    640.0,
    578.4368896484375,
    640.0,
    578.4368896484375,
    395.5082092285156]],
  'area': 88764.05484339781,
  'id': 1,
  'iscrowd': 0},
 {'image_id': 0,
  'category_id': 1,
  'bbox': [0.0, 249.67079162597656, 557.7630004882812, 390.32920837402344],
  'score': 0.061044514179229736,
  'segmentation': [[0.0,
    249.67079162597656,
    0.0,
    640.0,
    557.7630004882812,
    640.0,
    557.7630004882812,
    249.67079162597656]],
  'area': 217711.19044091087,
  'id': 2,
  'iscrowd': 0},
 {'image_id': 0,
  'category_id': 1,
  'bbox': [201.85061645507812,
   254.96240234375,
   342.8085632324219,
   359.4034423828125],
  'score': 0.0595320463180542,
  'segmentation': [[201.85061645507812,
    254.96240234375,
    201.85061

In [50]:
import copy
evaluator = CocoEvaluator(coco_gt=TEST_DATASET.coco, iou_types=["bbox"])

for data in nanodet_data:

    if data['score'] > 0.0:
        print(data)
        
        evaluator.update([data])

evaluator.synchronize_between_processes()
evaluator.accumulate()
evaluator.summarize()


{'image_id': 0, 'category_id': 2, 'bbox': [96.33306884765625, 404.11553955078125, 118.45657348632812, 40.0133056640625], 'score': 0.3991802930831909, 'segmentation': [[96.33306884765625, 404.11553955078125, 96.33306884765625, 444.12884521484375, 214.78964233398438, 444.12884521484375, 214.78964233398438, 404.11553955078125]], 'area': 4739.839082825929, 'id': 1, 'iscrowd': 0}
{'image_id': 0, 'category_id': 2, 'bbox': [59.22003173828125, 565.638671875, 99.22882080078125, 41.3145751953125], 'score': 0.24268300831317902, 'segmentation': [[59.22003173828125, 565.638671875, 59.22003173828125, 606.9532470703125, 158.4488525390625, 606.9532470703125, 158.4488525390625, 565.638671875]], 'area': 4099.596578516066, 'id': 1, 'iscrowd': 0}
{'image_id': 0, 'category_id': 2, 'bbox': [145.38497924804688, 473.3541564941406, 63.167022705078125, 29.90277099609375], 'score': 0.22241920232772827, 'segmentation': [[145.38497924804688, 473.3541564941406, 145.38497924804688, 503.2569274902344, 208.55200195312

In [48]:
image0 = [x for x in nanodet_data if x['image_id'] == 13]

draw_preds(13, image0, conf=0.3)

In [49]:
def draw_preds(image_id, detections, conf=0.25):
    input_pil_image, _ = TEST_DATASET[image_id]
    
    draw = ImageDraw.Draw(input_pil_image)

    # Define a color for each label
    colors = {1: "green", 2: "blue", 3: "red"}
    for detection in detections:
        if detection['score'] > conf:
            # Draw the bounding box on the image
            x1, y1, width, height = detection['bbox']
            
            draw.rectangle([x1, y1, x1+width, y1+height], outline=colors[int(detection['category_id'])], width=2)

    # Display the image
    input_pil_image.show()

In [82]:
calculate_metrics(inf_on_image, onnx_yolon)


Running evaluation...
0.80453384 1.0
0.43019015 1.0
0.15540627 2.0
0.09108779 1.0
0.05720362 1.0
0.055244952 1.0
0.054953188 1.0
0.04214111 1.0
0.03604743 1.0
0.02029711 1.0
0.019265175 1.0
0.018290311 1.0
0.018205285 1.0
0.015255034 1.0
0.014018834 1.0
0.013505399 1.0
0.013485581 1.0
0.013370097 2.0
0.010465056 1.0
0.010286629 1.0
0.010030866 1.0
0.007409185 1.0
0.0069520175 1.0
0.0067702234 1.0
0.0059640408 1.0
0.0047789514 1.0
0.00477013 1.0
0.0046601593 1.0
0.004436791 1.0
0.0042008758 2.0
0.004177153 2.0
0.003952205 2.0
0.0037287176 2.0
0.0035175383 0.0
0.0032535791 1.0
0.0029518008 1.0
0.0025835633 2.0
0.0025376081 1.0
0.0025102496 1.0
0.0024926066 1.0
0.0024467707 1.0
0.0024126768 2.0
0.0022917986 2.0
0.002221167 1.0
0.0020869374 1.0
0.0020593405 1.0
0.0020321608 1.0
0.001981467 1.0
0.001969397 1.0
0.001916647 1.0
0.0018888712 1.0
0.0018032491 1.0
0.0017979741 1.0
0.0017972589 1.0
0.0017921329 1.0
0.001513511 1.0
0.0015050769 2.0
0.0015002191 1.0
0.0014992952 1.0
0.0014522374 1.

In [58]:
test_img, _ = TEST_DATASET[2]

In [17]:
sess_options = rt.SessionOptions()
sess_options.graph_optimization_level = rt.GraphOptimizationLevel.ORT_ENABLE_ALL

# Specify DNNL as the execution provider
# session = rt.InferenceSession(yolon_path, sess_options, providers=['DnnlExecutionProvider'])
session = rt.InferenceSession(yolon_path, sess_options, providers=['CPUExecutionProvider'])

In [60]:
onnx_model = rt.InferenceSession(yolon_path)


In [61]:
onnx_model.get_inputs()[0].name

'images'

In [30]:
def benchmark_model_time(session, num=1000):
    total_time = 0.0
    num_runs = 30
    for i in range(0,num_runs):
        _, elp_time = inf_on_image(test_img, session)
        total_time += elp_time
    avg_time_ms = total_time / num_runs
    return avg_time_ms

In [68]:
avg_time_ms

61.02335736650275

In [62]:
qdr = QuantizationDataReader(TEST_DATASET, batch_size=1, input_name=session.get_inputs()[0].name)

In [46]:
qdr.get_next()

{'images': array([[[[0.01176471, 0.01176471, 0.01176471, ..., 0.02745098,
           0.02745098, 0.02745098],
          [0.01176471, 0.01176471, 0.01176471, ..., 0.02745098,
           0.02745098, 0.02745098],
          [0.01176471, 0.01176471, 0.01176471, ..., 0.02745098,
           0.02745098, 0.02745098],
          ...,
          [0.2627451 , 0.23921569, 0.21176471, ..., 0.12156863,
           0.10588235, 0.08627451],
          [0.24313726, 0.23137255, 0.21568628, ..., 0.12156863,
           0.10196079, 0.08627451],
          [0.21176471, 0.21176471, 0.21176471, ..., 0.12156863,
           0.10196079, 0.08627451]],
 
         [[0.03137255, 0.03137255, 0.03137255, ..., 0.04705882,
           0.04705882, 0.04705882],
          [0.03137255, 0.03137255, 0.03137255, ..., 0.04705882,
           0.04705882, 0.04705882],
          [0.03137255, 0.03137255, 0.03137255, ..., 0.04705882,
           0.04705882, 0.04705882],
          ...,
          [0.3137255 , 0.2901961 , 0.2627451 , ..., 0.133

In [51]:
yolon_320_path = "yolo320.onnx"

In [52]:
yolon320_path_prep = "yolo320_prep.onnx"
quantization.shape_inference.quant_pre_process(yolon_320_path, yolon320_path_prep, skip_symbolic_shape=False)

In [48]:
yolon_path_prep = "yolon_prep.onnx"
quantization.shape_inference.quant_pre_process(yolon_path, yolon_path_prep, skip_symbolic_shape=False)

In [63]:
q_static_opts = {"ActivationSymmetric":False,
                "WeightSymmetric":True}
if torch.cuda.is_available():
    q_static_opts = {"ActivationSymmetric":True,
                    "WeightSymmetric":True}

model_int8_path = 'yolon_int8.onnx'
quantized_model = quantization.quantize_static(model_input=yolon_path_prep,
                                               model_output=model_int8_path,
                                               calibration_data_reader=qdr)

In [64]:
int8_model = rt.InferenceSession(model_int8_path)

In [51]:
benchmark_model_time(int8_model, num=100)

134.7144072663165

In [68]:
inf_on_image(test_img,int8_model)[0][0][0]

array([[  0.       ,   0.       ,  17.800064 ,  14.833385 ,   0.       ,
          0.       ],
       [  0.       ,   0.       ,  17.800064 ,  14.833385 ,   0.       ,
          0.       ],
       [  0.       ,   0.       ,  17.800064 ,  14.833385 ,   0.       ,
          2.9666772],
       ...,
       [124.60044  ,  -2.9666772, 178.00063  ,  17.800064 ,   0.       ,
          0.       ],
       [124.60044  ,  -2.9666772, 178.00063  ,  17.800064 ,   0.       ,
          0.       ],
       [124.60044  ,  -2.9666772, 178.00063  ,  17.800064 ,   0.       ,
          2.9666772]], dtype=float32)

In [54]:
quantized_yolon_path = "yolon_int8_dyn.onnx"
quantized_yolon320_path = "yolon320_int8_dyn.onnx"

In [55]:
quantize_dynamic(yolon320_path_prep,
                     quantized_yolon320_path,
                     weight_type=QuantType.QUInt8)

In [129]:
from PIL import Image
def inf_on_image320(input_pil_image, onnx_model):
    input_pil_image = input_pil_image.resize((320, 320))
    #input_pil_image.show()
    image = np.array(input_pil_image)
    
    # Preprocess the images as required by your ONNX model
    # This might involve resizing, normalization, adding a batch dimension, etc.
    # The exact preprocessing steps will depend on your specific model

    # Convert the images to a format that can be used as input to the ONNX model
    # print(image)
    image = image.astype(np.float32) / 255.0
    # print(image)
    image = np.transpose(image, (2, 0, 1))  # Change data layout from HWC to CHW
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    print(image.shape)
    # Run the ONNX model
    input_name = onnx_model.get_inputs()[0].name

    start = time.perf_counter()
    predictions = onnx_model.run(None, {input_name: image})
    end = (time.perf_counter() - start) * 1000

    return predictions, end

In [81]:
int8_320_model_dyn = rt.InferenceSession(quantized_yolon320_path)
model320_prep = rt.InferenceSession(yolon320_path_prep)
model320 = rt.InferenceSession(yolon_320_path)
inf_on_image320(test_img,model320_prep)

(1, 3, 320, 320)


([array([[[2.3757764e+02, 2.3119797e+02, 2.5160287e+02, 2.4270418e+02,
           4.8794764e-01, 2.0000000e+00],
          [1.6449509e+02, 2.5057283e+02, 2.3316238e+02, 2.7691916e+02,
           3.7252486e-01, 1.0000000e+00],
          [2.5090421e+02, 2.7532578e+02, 2.9561163e+02, 2.9249689e+02,
           3.1467965e-01, 1.0000000e+00],
          ...,
          [3.5544395e-02, 2.5959354e+02, 2.8266857e+01, 2.8250555e+02,
           2.4151802e-04, 1.0000000e+00],
          [2.5180511e+00, 1.9731860e+02, 1.3757561e+02, 2.2084605e+02,
           2.3958087e-04, 1.0000000e+00],
          [2.6022387e-01, 2.5422995e+02, 1.7068537e+01, 2.9985529e+02,
           2.3850799e-04, 1.0000000e+00]]], dtype=float32)],
 21.96693400037475)

In [79]:
benchmark_model_time(model320_prep, num=100)

NameError: name 'benchmark_model_time' is not defined

In [130]:
calculate_metrics(inf_on_image320, model320, 2)

Running evaluation...
(1, 3, 320, 320)
0.71825004 1.0
0.15803051 2.0
0.084516704 1.0
0.079852134 1.0
0.068059355 1.0
0.064192384 1.0
0.033036053 1.0
0.0273785 2.0
0.026123703 1.0
0.020922154 2.0
0.016552567 2.0
0.01363188 1.0
0.013063848 1.0
0.012164414 1.0
0.011511356 1.0
0.009708732 1.0
0.009424925 2.0
0.008992642 2.0
0.00887993 2.0
0.007755339 1.0
0.0073926747 1.0
0.0063489974 1.0
0.0060496926 1.0
0.005663842 1.0
0.005348563 1.0
0.0048424304 1.0
0.0047410727 1.0
0.00469926 1.0
0.004179865 0.0
0.003966838 1.0
0.00390628 1.0
0.003838122 1.0
0.0037881434 2.0
0.003613174 2.0
0.003379196 1.0
0.003020674 2.0
0.0028714538 1.0
0.002699554 1.0
0.0026807785 2.0
0.0025846064 1.0
0.0022735596 1.0
0.00218305 1.0
0.0021388829 1.0
0.0020833313 1.0
0.0019432008 1.0
0.0018472672 1.0
0.0018056333 2.0
0.0017505884 2.0
0.0016625226 1.0
0.0016573668 1.0
0.0016441345 1.0
0.0015733838 1.0
0.001509577 1.0
0.0015049279 1.0
0.00144732 1.0
0.0014309287 1.0
0.0014276206 1.0
0.0014231801 2.0
0.0013251305 1.0
0.

In [131]:

calculate_metrics(inf_on_image320, int8_320_model_dyn, 2)

Running evaluation...
(1, 3, 320, 320)
0.66273296 1.0
0.12080702 2.0
0.09310803 1.0
0.072157264 1.0
0.048014402 1.0
0.033966243 1.0
0.032887608 1.0
0.02464351 2.0
0.019196987 2.0
0.018681794 1.0
0.016412944 2.0
0.015293181 1.0
0.009785593 2.0
0.008235455 1.0
0.0077337623 2.0
0.007580459 1.0
0.0075576305 1.0
0.007515222 1.0
0.007476181 1.0
0.0074344575 2.0
0.006590694 1.0
0.0061470866 1.0
0.0053399503 1.0
0.0051781535 2.0
0.004417181 1.0
0.0043724477 1.0
0.004172921 1.0
0.0038615465 1.0
0.0037181973 2.0
0.003603667 1.0
0.0034390986 1.0
0.0031196475 1.0
0.0029999018 1.0
0.0028253794 1.0
0.0027406812 2.0
0.0026631355 0.0
0.0024641156 1.0
0.0024023652 1.0
0.0023792386 2.0
0.0019027293 2.0
0.0017344654 2.0
0.0017052889 1.0
0.0016185641 1.0
0.0014999807 1.0
0.0014641881 2.0
0.0014638305 1.0
0.0013994873 1.0
0.0013599396 1.0
0.0013463199 1.0
0.0012978613 1.0
0.0012864172 1.0
0.0012846589 1.0
0.0012632906 2.0
0.0012506247 1.0
0.0012436509 1.0
0.00121665 1.0
0.0012030005 1.0
0.0011897683 2.0
0.

In [124]:
def draw_predsscale(input_pil_image, predictions, conf=0.70):
    draw = ImageDraw.Draw(input_pil_image)

    # Define a color for each label
    colors = {0: "green", 1: "blue", 2: "red"}

    for prediction in predictions[0][0]:

        x1, y1, x2, y2, score, label = prediction
        
        #print(score, label)

        if score > conf:
            # Draw the bounding box on the image
            x1, y1, x2, y2 = map(int, [x1*2, y1*2, x2*2, y2*2])
            draw.rectangle([x1, y1, x2, y2], outline=colors[int(label)], width=2)

    # Display the image
    input_pil_image.show()


In [125]:
im, _ = TEST_DATASET[5]
predictions, _ = inf_on_image320(im ,model320)

draw_predsscale(im, predictions, conf=0.1)

(1, 3, 320, 320)
